<a href="https://colab.research.google.com/github/TanzimRusho/AI-Resources/blob/master/New_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install textract

In [ ]:
!pip install nltk

In [ ]:
import PyPDF2
import textract
import re
import string

In [ ]:
import nltk
from nltk.corpus import words
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install nlp

In [ ]:
import os
import re
import subprocess
import glob
import io
import shutil
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tika

In [ ]:
from tika import parser

In [ ]:
folder_path = '/content/drive/MyDrive/-1.Rokomari_New/Backend_Senior/CVs/All_CVs'

In [ ]:
def is_english_word(word):
    return word.lower() in words.words()

def extract_non_english_words(text):
    tokens = nltk.word_tokenize(text)
    non_english_words = [word for word in tokens if not is_english_word(word)]
    return non_english_words

#Name
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)

    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

    matcher.add('NAME', [pattern], on_match = None)

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

In [ ]:
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S',
            # 'ME', 'M.E', 'M.E.', 'MS', 'M.S', 'MSC',  'M. SC. ENGG.', 'M.SC.', 'M.SC', 'M.Sc', 'M.TECH', 'MTECH',
            'BTECH', 'B.TECH',
            'B.Sc', 'B.SC.',  'BSc ENGINEERING', 'B.SC',
            'CBSE', 'ICSE', 'X', 'XII',
            'BSC', 'BBA', 'MBA',  'B. SC. ENGG.',
            'DIPLOMA IN ENGINEERING',
            'DIPLOMA ENGINEERING', 'BACHELOR OF SCIENCE',
          ]

def extract_degree(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    degree = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            degree.append((key, ''.join(year[0])))
        else:
            degree.append(key)
    return degree

def extract_university(resume_text):
    sub_patterns = ['[A-Z][a-z]* University','[A-Z][a-z]* Educational Institute',
                    'University of [A-Z][a-z]*', '[A-Z][a-z]* Institute',
                    '[A-Z][a-z]* [A-Z][a-z]* Institute',
                    '[A-Z][a-z]* [A-Z][a-z]* University',
                    '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University',
                    '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University',
                    '[A-Z][a-z]* University [A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]*',
                    ]

    univ_pattern = '({})'.format('|'.join(sub_patterns))
    university = re.findall(univ_pattern, text)
    return university

def extract_experience(resume_text):
    sub_patterns = ['[A-Z][a-z]* [A-Z][a-z]* Private Limited','[A-Z][a-z]* [A-Z][a-z]* Pvt. Ltd.','[A-Z][a-z]* [A-Z][a-z]* Inc.',
                    '[A-Z][a-z]* [A-Z][a-z]* Limited',  '[A-Z][a-z]* [A-Z][a-z]*  (Pvt.) Ltd.',   '[A-Z][a-z]* [A-Z][a-z]*  Ltd.',
                    '[A-Z][a-z]* [A-Z][a-z]* Ltd',
                    '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* Ltd.', '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* Ltd',
                    '[A-Z][a-z]* [A-Z][a-z]* LLC', '[A-Z][a-z]* LLC', '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* Limited',
                  ]
    exp_pattern = '({})'.format('|'.join(sub_patterns))

    Exp = re.findall(exp_pattern, text)

    return Exp

In [ ]:
df = pd.DataFrame(columns=['Count', 'Filename', 'Candidate', 'Degree', 'University',
                           'Experience', 'Score'])

file_count = 0

for filename in glob.glob(os.path.join(folder_path, '*.pdf')):
        with open(os.path.join(os.getcwd(), filename), 'r', encoding="utf8", errors='ignore') as f:
            try:
                file_count += 1
                print("File ", file_count, end=', ')

                # Using PyPDF2
                # pdfFileObj = open(filename, 'rb')
                # pdfReader = PyPDF2.PdfReader(pdfFileObj)
                # num_pages = len(pdfReader.pages)

                # # print(filename[-30:-4], end=', ')

                # count = 0

                # text = ""

                # while count < num_pages:
                #     pageObj = pdfReader.pages[count]
                #     count += 1
                #     text += pageObj.extract_text()

                # text = text.lower()

                # text = re.sub(r'\d+', '', text)

                # text = text.translate(str.maketrans('', '', string.punctuation))

                # Copied from kaggle_parser
                file_data = parser.from_file(filename)
                text = file_data['content']

                # Name part starts from here

                # name = text[:30]

                # non_english_words = extract_non_english_words(name)

                # new_name = ""

                # for non_english_word in non_english_words[:3]:
                #     if non_english_word in ['vitae', 'VITAE', 'Vitae', '.', ',']:
                #         continue
                #     new_name += non_english_word + " "

                # # print(new_name, end=', ')

                # Name part ends here

                name = extract_name(text)

                degree = extract_degree(text)

                university = extract_university(text)

                experience = extract_experience(text)

                terms = {
                        'Backend':['mysql', 'mongodb', 'backend', 'intern', 'html',
                                  'css', 'microservices', 'mvc', 'clean', 'java',
                                  'spring', 'python', 'django', 'golang', 'database',
                                  'algorithm', 'contest', 'icpc', 'regional',
                                  'leetcode', 'codeforces', 'codechef', 'hackerrank',
                                  'uva', 'stopstalk', 'soap', 'restapi', 'postgresql',
                                  'nosql', 'github', 'git', 'problem', 'solving',
                                  'programming', 'aws', 'azure', 'cloud', 'machine',
                                  'learning', 'research', 'docker', 'kubernetes',
                                  'javascript', 'agile', 'scrum', 'ajax', 'json',
                                  'kafka', 'redis', 'scalable', 'gateway', 'api',
                                  'nginx', 'linux', 'token', 'jwt', 'authentication',
                                  'security', 'artificial', 'intelligence', 'hibernate',
                                  'jsp', 'c#', 'wpf', 'jquery', 'flask', 'jenkins',
                                  'restful', 'web', 'sqlite', 'bootstrap', 'cluster',
                                  'ubuntu', 'cli', 'springboot', 'rabbitmq', 'debugging',
                                  'asp.net', '.net', 'gorilla', 'gin', 'fiber',
                                  'gitlab', 'cassandra', 'websocket', 'webrtc',
                                  'caching', 'queueing', 'message', 'cache', 'queue',
                                  'orm', 'tcp', 'udp', 'structures',
                                  'request', 'bash', 'powershell', 'stackoverflow',
                                  'design', 'acm', 'ncpc', 'pattern', 'circuit',
                                  'jdbc', 'logging', 'authorization', 'linq',
                                  'entity','solid', 'dry', 'graphql', 'oop', 'postman',
                                  'ci/cd', 'socket.io', 'gitops', 's3', 'dynamodb',
                                  'node.js', 'oracle',
                                  ],
                }

                score = 0

                for area in terms.keys():
                    if area == 'Backend':
                        for word in terms[area]:
                            if word in text:
                                score += 1

                # print(sqa, end=', ')

                list1 = []

                list1.append(file_count)
                list1.append(filename)
                # list1.append(new_name)
                list1.append(name)
                list1.append(degree)
                list1.append(university)
                list1.append(experience)
                list1.append(score)

                print(list1)

                df.loc[len(df)] = list1

            except IndexError:
                continue

# file_count, filename, new_name, sqa

File  1, [1, '/content/drive/MyDrive/-1.Rokomari_New/Backend_Senior/CVs/All_CVs/sourav_ahmed.pdf', 'Microsoft Word', [], ['Daffodil International University'], [('T Bangla Ltd', '')], 13]
File  2, [2, '/content/drive/MyDrive/-1.Rokomari_New/Backend_Senior/CVs/All_CVs/Mahmaud Al Hossain.pdf', 'Mahmud Al', [('BSC', '2019')], ['Southeast University', 'Agricultural University'], [('Unisoft Systems Limited', '')], 9]
File  3, [3, '/content/drive/MyDrive/-1.Rokomari_New/Backend_Senior/CVs/All_CVs/Ariful-cv.pdf', 'Islam Anni', [('BSc', '2023')], ['World University', 'Engineering University'], [], 6]
File  4, [4, '/content/drive/MyDrive/-1.Rokomari_New/Backend_Senior/CVs/All_CVs/AL AMIN.pdf', 'AL AMIN', [('Bsc', '2021')], [], [], 12]
File  5, [5, '/content/drive/MyDrive/-1.Rokomari_New/Backend_Senior/CVs/All_CVs/Resume_of_Sohanur_Rahman.pdf', 'Java Developer', [], [], [('Tell Consulting Ltd', ''), ('Tell Consulting Ltd', '')], 5]
File  6, [6, '/content/drive/MyDrive/-1.Rokomari_New/Backend_Sen

In [ ]:
df

,Count,Filename,Candidate,Degree,University,Experience,Score
0,1,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Microsoft Word,[],[Daffodil International University],"[(T Bangla Ltd, )]",13
1,2,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Mahmud Al,"[(BSC, 2019)]","[Southeast University, Agricultural University]","[(Unisoft Systems Limited, )]",9
2,3,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Islam Anni,"[(BSc, 2023)]","[World University, Engineering University]",[],6
3,4,/content/drive/MyDrive/-1.Rokomari_New/Backend...,AL AMIN,"[(Bsc, 2021)]",[],[],12
4,5,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Java Developer,[],[],"[(Tell Consulting Ltd, ), (Tell Consulting Ltd...",5
...,...,...,...,...,...,...,...
90,94,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Computer Science,"[(BSc, 2020)]",[American International University],[],17
91,97,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Modern Minimalist,[],[C University],[],12
92,98,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Mechine Learning,[],"[University of Dhaka, In Southeast University,...","[(T It Bangla Ltd,, ), (Organized By Banglades...",11
93,99,/content/drive/MyDrive/-1.Rokomari_New/Backend...,Asad Miah,"[(Bsc, 2023)]",[],[],12


In [ ]:
outdir = folder_path + '/result'

if not os.path.exists(outdir):
    os.mkdir(outdir)

my_path = os.path.abspath(outdir)
my_file = 'candidates.csv'

df.to_csv(os.path.join(my_path, my_file), index=False)

In [ ]:
# my_path = os.path.abspath(folder_path)
# my_file = 'backend_candidates.csv'

# df.to_csv(os.path.join(my_path, my_file), index=False)